# SURVIVAL PREDICTION IN BREAST CANCER USING GENE EXPRESSION AND DIAGNOSES DATA

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv('final_tpm_unstranded.tsv', sep='\t')

/tmp/ipykernel_101742/2822957246.py:1: DtypeWarning: Columns (19944,19951,19952,19957,19958,19974,20003,20004,20008,20104) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('final_tpm_unstranded.tsv', sep='\t')


In [3]:
df.head(2)

,Unnamed: 0,file_id,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,...,treatments.treatment_duration,treatments.treatment_effect,treatments.treatment_effect_indicator,treatments.treatment_frequency,treatments.treatment_id,treatments.treatment_intent_type,treatments.treatment_or_therapy,treatments.treatment_outcome,treatments.treatment_outcome_duration,treatments.treatment_type
0,0,56be281f-e85c-4dcb-9d75-cd9ae3430bbe,0.2784,0.0064,185.8713,0.0691,0.0,21.2918,0.1043,33.0771,...,'--,'--,'--,'--,3234723f-c2c7-4b8b-a889-8ab985f2bf8d,'--,no,'--,'--,"Pharmaceutical Therapy, NOS"
1,1,56be281f-e85c-4dcb-9d75-cd9ae3430bbe,0.2784,0.0064,185.8713,0.0691,0.0,21.2918,0.1043,33.0771,...,'--,'--,'--,'--,e0a284cf-2fd8-46a1-a7af-7baf53c1b1cc,'--,no,'--,'--,"Radiation Therapy, NOS"


## DATAFRAME CLEANING

In [4]:
df.replace("'--", np.nan, inplace=True)

/tmp/ipykernel_101742/810073034.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("'--", np.nan, inplace=True)


In [5]:
df.dropna(how='all', inplace=True, axis=1)

In [6]:
df.drop('treatments.submitter_id', inplace=True, axis=1)

In [7]:
df.shape

(6013, 20014)

In [8]:
df['treatments.days_to_treatment_end'] = df['treatments.days_to_treatment_end'].astype(float)

In [9]:
cases = []
diagnoses = []
treatments = []
demographics = []
genes = []

for col in df.columns:
    if col.startswith('diagn'):
        diagnoses.append(col)
    elif 'treat' in col[:7]:
        treatments.append(col)
    elif col.startswith('case'):
        cases.append(col)
    elif col.startswith('demog'):
        demographics.append(col)
    else:
        genes.append(col)

columns = {
    'cases': cases,
    'treatments': treatments,
    'demographics': demographics,
    'genes': genes,
    'diagnoses': diagnoses
}

"""
"""

del diagnoses
del cases
del treatments
del genes
del demographics

In [10]:
len(columns['genes'])

19941

In [11]:
df[df['treatments.days_to_treatment_end'] < 0].shape

(8, 20014)

In [12]:
df = df[df['treatments.days_to_treatment_end'] >= 0]

### SURVIVAL DATA

In [13]:
df['treatments.days_to_treatment_end'].sample(3)

4653    242.0
5487    244.0
3053    135.0
Name: treatments.days_to_treatment_end, dtype: float64

In [14]:
df['treatments.days_to_treatment_end'].describe()

count    2634.000000
mean      311.921412
std       569.088844
min         0.000000
25%       131.000000
50%       181.000000
75%       257.750000
max      6552.000000
Name: treatments.days_to_treatment_end, dtype: float64

## FEATURE SELECTION

###  BREAST CANCER GENES

In [15]:
from bioservices import KEGG

In [16]:
kegg = KEGG()

WARNING [bioservices.KEGG:130]:  The URL (http://rest.kegg.jp) provided cannot be reached.


In [17]:
results = kegg.find("pathway", "breast cancer")
print(results)

path:map05224	Breast cancer



In [18]:
import requests

url = "https://rest.kegg.jp/link/hsa/hsa05224"
response = requests.get(url)


In [19]:
genes = [line.split("\t")[1].replace("hsa:", "")
         for line in response.text.strip().split("\n")]

In [122]:
with open("breast_cancer_genes.txt", "a") as f:
    f.write(str(genes))

In [20]:
import mygene

mg = mygene.MyGeneInfo()

gene_ids = genes
results = mg.querymany(
    gene_ids,
    scopes="entrezgene",
    fields="symbol",
    species="human"
)

symbols = {r["query"]: r.get("symbol") for r in results}
print(symbols)


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed


{'10000': 'AKT3', '10023': 'FRAT1', '1019': 'CDK4', '1021': 'CDK6', '1026': 'CDKN1A', '10297': 'APC2', '10683': 'DLL3', '10912': 'GADD45G', '110117499': 'P3R3URF-PIK3R3', '11211': 'FZD10', '122011': 'CSNK1A1L', '1452': 'CSNK1A1', '1499': 'CTNNB1', '1643': 'DDB2', '1647': 'GADD45A', '182': 'JAG1', '1855': 'DVL1', '1856': 'DVL2', '1857': 'DVL3', '1869': 'E2F1', '1870': 'E2F2', '1871': 'E2F3', '1950': 'EGF', '1956': 'EGFR', '2064': 'ERBB2', '207': 'AKT1', '208': 'AKT2', '2099': 'ESR1', '2100': 'ESR2', '2246': 'FGF1', '2247': 'FGF2', '2248': 'FGF3', '2249': 'FGF4', '2250': 'FGF5', '2251': 'FGF6', '2252': 'FGF7', '2253': 'FGF8', '2254': 'FGF9', '2255': 'FGF10', '2260': 'FGFR1', '2324': 'FLT4', '23401': 'FRAT2', '23462': 'HEY1', '23493': 'HEY2', '2353': 'FOS', '2475': 'MTOR', '2535': 'FZD2', '25759': 'SHC2', '26281': 'FGF20', '26291': 'FGF21', '26508': 'HEYL', '27006': 'FGF22', '28514': 'DLL1', '2885': 'GRB2', '2932': 'GSK3B', '324': 'APC', '3265': 'HRAS', '3280': 'HES1', '3479': 'IGF1', '34

In [21]:
breast_cancer_genes = list(symbols.values())
gene_columns = breast_cancer_genes

In [22]:
len(breast_cancer_genes)

148

In [23]:
for i in df.columns:
    if 'typ' in i:
        print(i)

cases.consent_type
cases.disease_type
treatments.treatment_intent_type
treatments.treatment_type


### TREATMENTS FEATURES

In [24]:
treatment_columns = columns['treatments']
str(treatment_columns)

"['treatments.clinical_trial_indicator', 'treatments.course_number', 'treatments.days_to_treatment_end', 'treatments.days_to_treatment_start', 'treatments.initial_disease_status', 'treatments.margin_status', 'treatments.number_of_cycles', 'treatments.number_of_fractions', 'treatments.prescribed_dose', 'treatments.prescribed_dose_units', 'treatments.regimen_or_line_of_therapy', 'treatments.route_of_administration', 'treatments.therapeutic_agents', 'treatments.treatment_anatomic_sites', 'treatments.treatment_dose', 'treatments.treatment_dose_units', 'treatments.treatment_id', 'treatments.treatment_intent_type', 'treatments.treatment_or_therapy', 'treatments.treatment_outcome', 'treatments.treatment_type']"

In [25]:
len(treatment_columns)

21

In [26]:
treatment_f = ['treatments.days_to_treatment_start', 'treatments.initial_disease_status',
               'treatments.therapeutic_agents', # REQUIRED ENGINEERING
                'treatments.treatment_anatomic_sites', 'treatments.treatment_dose_units',# REQUIRED ENGINEERING
                'treatments.treatment_or_therapy'
            ] # FEATURES TO BE USED

treatment_l = ['treatments.treatment_outcome'] # POSSIBLE LABELS
def treatment_x(): # EXCLUDED
    """
        Excluded columns in training
    """
    return [ x for x in treatment_columns if x not in treatment_f ]
#str(treatment_x())

In [27]:
df[['case_id', 'treatments.treatment_outcome']+treatment_f].head(5)

,case_id,treatments.treatment_outcome,treatments.days_to_treatment_start,treatments.initial_disease_status,treatments.therapeutic_agents,treatments.treatment_anatomic_sites,treatments.treatment_dose_units,treatments.treatment_or_therapy
3,02f5ae33-a563-4ecb-9e33-dfa500a44931,Complete Response,28,NaN,NaN,Primary Tumor Field,NaN,yes
8,74f31744-0aed-4633-beec-4e203315f0b7,Complete Response,118,NaN,Docetaxel,NaN,NaN,yes
10,74f31744-0aed-4633-beec-4e203315f0b7,Complete Response,118,NaN,Cyclophosphamide,NaN,NaN,yes
11,74f31744-0aed-4633-beec-4e203315f0b7,Complete Response,118,NaN,Doxorubicin Hydrochloride,NaN,NaN,yes
15,c2aeee6c-ba10-4cf9-b560-c739580f7bf3,NaN,164,NaN,Docetaxel,NaN,NaN,yes


In [28]:
df['treatments.treatment_outcome'].value_counts()

treatments.treatment_outcome
Complete Response      796
Unknown                136
Progressive Disease     69
Stable Disease          45
Partial Response        16
Name: count, dtype: int64

In [29]:
df['treatments.therapeutic_agents'].value_counts()

treatments.therapeutic_agents
Cyclophosphamide                                 532
Paclitaxel                                       231
Doxorubicin                                      207
Docetaxel                                        203
Doxorubicin Hydrochloride                        174
Fluorouracil                                     106
Tamoxifen                                         82
Trastuzumab                                       78
Anastrozole                                       68
Epirubicin                                        39
Carboplatin                                       35
Methotrexate                                      32
Exemestane                                        20
Letrozole                                         19
Bevacizumab                                       18
Nab-paclitaxel                                    15
Capecitabine                                      13
Zoledronic Acid                                    8
Pegfilgrastim   

In [30]:
df['treatments.margin_status'].value_counts()

Series([], Name: count, dtype: int64)

In [31]:
df[df['treatments.treatment_outcome'] == 'Complete Response'].head(2)[['cases.case_id', 'treatments.treatment_outcome']]

,cases.case_id,treatments.treatment_outcome
3,02f5ae33-a563-4ecb-9e33-dfa500a44931,Complete Response
8,74f31744-0aed-4633-beec-4e203315f0b7,Complete Response


In [32]:
df[df["cases.case_id"] == '02f5ae33-a563-4ecb-9e33-dfa500a44931'][['A1BG','cases.case_id', 'treatments.treatment_outcome']]

,A1BG,cases.case_id,treatments.treatment_outcome
3,0.2784,02f5ae33-a563-4ecb-9e33-dfa500a44931,Complete Response


In [33]:
treatment_l 

['treatments.treatment_outcome']

In [34]:
df['treatments.treatment_outcome'].value_counts()

treatments.treatment_outcome
Complete Response      796
Unknown                136
Progressive Disease     69
Stable Disease          45
Partial Response        16
Name: count, dtype: int64

In [35]:
df['treatments.treatment_outcome'].value_counts()

treatments.treatment_outcome
Complete Response      796
Unknown                136
Progressive Disease     69
Stable Disease          45
Partial Response        16
Name: count, dtype: int64

In [36]:
#df['treatments.route_of_administration'].sample(4)

### DIAGNOSES FEATURES

In [37]:
diagnoses_columns = columns['diagnoses']
diagnoses_f = ['diagnoses.age_at_diagnosis','diagnoses.ajcc_staging_system_edition', 'diagnoses.ajcc_pathologic_m', 'diagnoses.ajcc_pathologic_n', 
                       'diagnoses.ajcc_pathologic_t', 'diagnoses.classification_of_tumor', 'diagnoses.diagnosis_is_primary_disease', 
                      #'diagnoses.figo_stage',   MOSTLY NANs
                     'diagnoses.icd_10_code', 'diagnoses.laterality', 'diagnoses.metastasis_at_diagnosis',
                      #'diagnoses.method_of_diagnosis',# MIGHT REMOVE THIS
                       'diagnoses.morphology', 'diagnoses.primary_diagnosis', 'diagnoses.prior_malignancy', 'diagnoses.prior_treatment',
                      'diagnoses.site_of_resection_or_biopsy', 'diagnoses.sites_of_involvement', 'diagnoses.synchronous_malignancy', 
                       'diagnoses.tissue_or_organ_of_origin', 'diagnoses.tumor_grade', 'diagnoses.tumor_of_origin', 'diagnoses.year_of_diagnosis'    
              ] # FEATURES TO BE USED

diagnoses_l = ['diagnoses.last_known_disease_status', 'diagnoses.progression_or_recurrence'] # POSSIBLE LABELS
def diagnoses_x(): # EXCLUDED
    """
        Excluded columns in training
    """
    return [ x for x in diagnoses_columns if x not in diagnoses_f ]
str(diagnoses_x())

"['diagnoses.ajcc_pathologic_stage', 'diagnoses.days_to_diagnosis', 'diagnoses.days_to_last_follow_up', 'diagnoses.diagnosis_id', 'diagnoses.figo_stage', 'diagnoses.figo_staging_edition_year', 'diagnoses.last_known_disease_status', 'diagnoses.method_of_diagnosis', 'diagnoses.progression_or_recurrence', 'diagnoses.submitter_id']"

In [38]:
str(diagnoses_columns)

"['diagnoses.age_at_diagnosis', 'diagnoses.ajcc_pathologic_m', 'diagnoses.ajcc_pathologic_n', 'diagnoses.ajcc_pathologic_stage', 'diagnoses.ajcc_pathologic_t', 'diagnoses.ajcc_staging_system_edition', 'diagnoses.classification_of_tumor', 'diagnoses.days_to_diagnosis', 'diagnoses.days_to_last_follow_up', 'diagnoses.diagnosis_id', 'diagnoses.diagnosis_is_primary_disease', 'diagnoses.figo_stage', 'diagnoses.figo_staging_edition_year', 'diagnoses.icd_10_code', 'diagnoses.last_known_disease_status', 'diagnoses.laterality', 'diagnoses.metastasis_at_diagnosis', 'diagnoses.method_of_diagnosis', 'diagnoses.morphology', 'diagnoses.primary_diagnosis', 'diagnoses.prior_malignancy', 'diagnoses.prior_treatment', 'diagnoses.progression_or_recurrence', 'diagnoses.site_of_resection_or_biopsy', 'diagnoses.sites_of_involvement', 'diagnoses.submitter_id', 'diagnoses.synchronous_malignancy', 'diagnoses.tissue_or_organ_of_origin', 'diagnoses.tumor_grade', 'diagnoses.tumor_of_origin', 'diagnoses.year_of_diag

In [39]:
df['diagnoses.last_known_disease_status'].value_counts()

Series([], Name: count, dtype: int64)

In [40]:
df['diagnoses.last_known_disease_status'].isna().sum()

np.int64(2634)

### DEMOGRAPHICS FEATURES

In [41]:
demographics_columns = columns['demographics']

In [42]:
len(demographics_columns)

12

In [43]:
str(demographics_columns)

"['demographic.age_at_index', 'demographic.age_is_obfuscated', 'demographic.country_of_residence_at_enrollment', 'demographic.days_to_birth', 'demographic.days_to_death', 'demographic.demographic_id', 'demographic.ethnicity', 'demographic.gender', 'demographic.race', 'demographic.submitter_id', 'demographic.vital_status', 'demographic.year_of_birth']"

In [44]:
df['survival'] = (df['demographic.vital_status'] == "Alive").astype(int)
df['survival'].value_counts()

survival
1    2341
0     293
Name: count, dtype: int64

In [45]:
demographics_f = ['demographic.age_at_index','demographic.gender','demographic.race']
demographics_l = ['survival']

In [46]:
df[demographics_f].sample(2)

,demographic.age_at_index,demographic.gender,demographic.race
102,47,female,white
968,40,female,white


In [47]:
(df[df['survival'] == 0][['case_id', 'survival']].groupby('case_id').max() - df[df['survival'] == 0][['case_id', 'survival']].groupby('case_id').min())['survival'].value_counts()

survival
0    58
Name: count, dtype: int64

In [48]:
df['demographic.gender'].value_counts()

demographic.gender
female    2610
male        24
Name: count, dtype: int64

In [49]:
df['demographic.vital_status'].value_counts()

demographic.vital_status
Alive    2341
Dead      293
Name: count, dtype: int64

###  CASES FEATURES

In [50]:
df['cases.disease_type'].value_counts()

cases.disease_type
Ductal and Lobular Neoplasms             2564
Complex Epithelial Neoplasms               41
Cystic, Mucinous and Serous Neoplasms      10
Epithelial Neoplasms, NOS                   7
Basal Cell Neoplasms                        5
Adnexal and Skin Appendage Neoplasms        4
Adenomas and Adenocarcinomas                2
Squamous Cell Neoplasms                     1
Name: count, dtype: int64

In [51]:
cases_f = ['cases.disease_type', ]

##  DIRTY PREPROCESSING

In [52]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

### DIAGNOSES COLUMNS

In [53]:
diagnoses_categorical_columns = ['diagnoses.ajcc_pathologic_m', 'diagnoses.ajcc_pathologic_n', 
                                 'diagnoses.ajcc_pathologic_t', 'diagnoses.classification_of_tumor', 
                                'diagnoses.icd_10_code', 'diagnoses.laterality', 'diagnoses.metastasis_at_diagnosis',
                                'diagnoses.method_of_diagnosis', 
                              'diagnoses.morphology', 'diagnoses.prior_malignancy', 'diagnoses.prior_treatment']


In [54]:
df['diagnoses.progression_or_recurrence'].value_counts()

Series([], Name: count, dtype: int64)

In [55]:
df['diagnoses.diagnosis_is_primary_disease'].value_counts()

diagnoses.diagnosis_is_primary_disease
True    2591
true      43
Name: count, dtype: int64

In [56]:
df['diagnoses.diagnosis_is_primary_disease'] = (df['diagnoses.diagnosis_is_primary_disease'].replace({"true": True}).replace({"false": False})).astype(float)

/tmp/ipykernel_101742/3793047122.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['diagnoses.diagnosis_is_primary_disease'] = (df['diagnoses.diagnosis_is_primary_disease'].replace({"true": True}).replace({"false": False})).astype(float)


In [57]:
#df['diagnoses.diagnosis_is_primary_disease']

In [58]:
df['diagnoses.age_at_diagnosis'].isna().sum()
df['diagnoses.age_at_diagnosis'] = df['diagnoses.age_at_diagnosis'].astype(float)

In [59]:
df['age_missing'] = df['diagnoses.age_at_diagnosis'].apply(lambda x: int(x is np.nan))

#df[['age_missing','diagnoses.age_at_diagnosis']]

In [60]:
diagnoses_numerical_columns = [ 'age_missing', 'diagnoses.diagnosis_is_primary_disease']

In [61]:
df[diagnoses_numerical_columns].astype(float).sample(3)

,age_missing,diagnoses.diagnosis_is_primary_disease
2907,0.0,1.0
3317,0.0,1.0
5715,0.0,1.0


In [62]:
diagnoses_generated_columns = ['age_missing']

### TREATMENT COLUMNS

In [63]:
str(treatment_f)

"['treatments.days_to_treatment_start', 'treatments.initial_disease_status', 'treatments.therapeutic_agents', 'treatments.treatment_anatomic_sites', 'treatments.treatment_dose_units', 'treatments.treatment_or_therapy']"

In [64]:
treatment_categorical_columns = ['treatments.initial_disease_status', 'treatments.treatment_or_therapy']

In [65]:
treatment_numerical_columns = ['treatments.days_to_treatment_start', ]

In [66]:
treatment_engineered_columns = ['treatments.therapeutic_agents', 'treatments.treatment_anatomic_sites']

In [67]:
df['treatments.days_to_treatment_start'].astype(float).sum()

np.float64(502641.0)

In [68]:
df['treatments.regimen_or_line_of_therapy'].isna().sum()

np.int64(2627)

In [69]:
df[treatment_f].sample(3)

,treatments.days_to_treatment_start,treatments.initial_disease_status,treatments.therapeutic_agents,treatments.treatment_anatomic_sites,treatments.treatment_dose_units,treatments.treatment_or_therapy
2643,80,NaN,Doxorubicin Hydrochloride,NaN,NaN,yes
783,82,NaN,Cyclophosphamide,NaN,NaN,yes
541,98,NaN,Trastuzumab,NaN,NaN,yes


###  DEMOGRAPHIC COLUMNS

In [70]:
demographics_f

['demographic.age_at_index', 'demographic.gender', 'demographic.race']

In [71]:
df['demographic.age_at_index'].isna().sum()

np.int64(0)

In [72]:
df['demographic.age_at_index'] = df['demographic.age_at_index'].astype(float)

In [73]:
df['demographic.gender'] = df['demographic.gender'].map({"female": 1, "male": 0})

In [74]:
df['demographic.race'].value_counts()

demographic.race
white                               1927
black or african american            473
not reported                         157
asian                                 74
american indian or alaska native       3
Name: count, dtype: int64

In [75]:
demographics_categorical_columns = ['demographic.race']

In [76]:
demographics_numerical_columns = ['demographic.age_at_index', 'demographic.gender']

##  CLEAN PREPROCESSING

In [77]:
numerical_features = diagnoses_numerical_columns + treatment_numerical_columns + demographics_numerical_columns
numerical_features_base = diagnoses_numerical_columns +  demographics_numerical_columns

In [78]:
categorical_features = diagnoses_categorical_columns + treatment_categorical_columns + demographics_categorical_columns
categorical_features_base = diagnoses_categorical_columns +  demographics_categorical_columns

In [79]:
#df.to_csv("cleaned_data.tsv", sep='\t')

### COLUMN TRANSFORMERS

In [80]:
numerical_transformer = Pipeline(
    steps=[
        ("scaler", StandardScaler())
    ]
)

In [81]:
categorical_transformer = Pipeline(
    steps=[
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ]
)

In [82]:
gene_transformer = Pipeline(
    steps=[
        ("logtransform", FunctionTransformer(np.log1p)),
        ("scaler", StandardScaler())
    ]
)

In [83]:
preprocessor = ColumnTransformer(
        transformers=[
            ("gene", gene_transformer, gene_columns),
            ("cat", categorical_transformer, categorical_features),
            ("num", numerical_transformer, numerical_features)            
        ],
        remainder="drop"
)
preprocessor_base = ColumnTransformer(
        transformers=[
            ("gene", gene_transformer, gene_columns),
            ("cat", categorical_transformer, categorical_features_base),
            ("num", numerical_transformer, numerical_features_base)            
        ],
        remainder="drop"
)
preprocessor_gene = ColumnTransformer(
        transformers=[
            ("gene", gene_transformer, gene_columns),          
        ],
        remainder="drop"
)
preprocessor_cox = ColumnTransformer(
        transformers=[
            ("num", numerical_transformer, diagnoses_numerical_columns),
            #("cat", categorical_transformer, diagnoses_categorical_columns),            
        ],
        remainder="drop"
)
preprocessor_rsf = ColumnTransformer(
        transformers=[
            ("num", numerical_transformer, diagnoses_numerical_columns),
            ("cat", categorical_transformer, diagnoses_categorical_columns),            
        ],
        remainder="drop"
)
preprocessor_rsf_gene = ColumnTransformer(
        transformers=[
            ("num", numerical_transformer, gene_columns)           
        ],
        remainder="drop"
)
preprocessor_rsf_multi = ColumnTransformer(
        transformers=[
            ("num", numerical_transformer, diagnoses_numerical_columns+gene_columns),
            ("cat", categorical_transformer, diagnoses_categorical_columns),            
        ],
        remainder="drop"
)

In [84]:
#X_base = df[gene_columns + diagnoses_f + demographics_f]
#X_gene = df[gene_columns]
#X_treat = df[gene_columns + diagnoses_f + treatment_f + demographics_f]
#y_survival = df['survival']
#y_base = df['survival']
#y_treat = df['survival']
#base = df[list(X_base.columns) + ['survival']]
#treat = df[list(X_treat.columns) + ['survival']]
#gene = df[list(X_gene.columns) + ['survival']]

## MODEL DEVELOPMENT

In [85]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as RFC, RandomForestRegressor as RFR
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

In [86]:
df["time"] = df["treatments.days_to_treatment_end"]
df["event"] = df["demographic.vital_status"].map({"Alive": 0, "Dead": 1})

In [87]:
print(df["event"].value_counts())
print(df["time"].isna().sum())

event
0    2341
1     293
Name: count, dtype: int64
0


### COX

In [88]:
from sksurv.linear_model import CoxPHSurvivalAnalysis as CoxPH
from sksurv.metrics import concordance_index_censored as c_index
from scipy import sparse

In [89]:
df_cox = df[~df['time'].isna()]

In [90]:
X_cox = preprocessor_cox.fit_transform(df_cox)
X_cox_feature_names = preprocessor_cox.get_feature_names_out()


if sparse.issparse(X_cox):
    X_cox = X_cox.toarray()

X_cox_df = pd.DataFrame(
    X_cox,
    columns=X_cox_feature_names,
    index=df_cox.index
)
X_cox_df['time'] = df_cox['time']
X_cox_df['event'] = df_cox['event']

In [91]:
coxph = CoxPH()

y_cox = np.array(
    [(bool(e), t) for e, t in zip(X_cox_df["event"], X_cox_df["time"])],
    dtype=[("event", bool), ("time", float)]
)

In [92]:
n_events = y_cox["event"].sum()
n_features = X_cox_df.shape[1]

n_events, n_features


(np.int64(293), 4)

In [93]:
estimator = coxph.fit(X_cox_df, y_cox)

LinAlgError: Matrix is singular.

In [95]:
chf_funcs = estimator.predict_cumulative_hazard_function(X_cox[:10])

NameError: name 'estimator' is not defined

In [ ]:
y_cox.dtype

In [ ]:
np.unique(y_cox["event"])

In [ ]:
risk = coxph.predict(X_cox)

In [ ]:

cindex = concordance_index_censored(
    y_cox["event"],
    y_cox["time"],
    risk
)[0]

cindex

In [ ]:

X_cox_train, X_cox_test, y_cox_train, y_cox_test = train_test_split(
    X_cox, y_cox, test_size=0.3, random_state=42
)

#coxph = CoxPHSurvivalAnalysis()
coxph.fit(X_cox_train, y_cox_train)

risk_test = coxph.predict(X_test)

cindex_test = concordance_index_censored(
    y_cox_test["event"],
    y_cox_test["time"],
    risk_test
)[0]

cindex_test


### RSF

In [98]:
from sksurv.ensemble import RandomSurvivalForest as RSF

In [99]:
df_rsf = df[~df['time'].isna()]

#### RSF DIAGNOSES

In [100]:
X_rsf = preprocessor_rsf.fit_transform(df_rsf)
X_rsf_feature_names = preprocessor_rsf.get_feature_names_out()


if sparse.issparse(X_rsf):
    X_rsf = X_rsf.toarray()

X_rsf_df = pd.DataFrame(
    X_rsf,
    columns=X_rsf_feature_names,
    index=df_rsf.index
)
X_rsf_df['time'] = df_rsf['time']
X_rsf_df['event'] = df_rsf['event']

y_rsf = np.array(
    [(bool(e), t) for e, t in zip(X_rsf_df["event"], X_rsf_df["time"])],
    dtype=[("event", bool), ("time", float)]
)

In [101]:
X_rsf_train, X_rsf_test, y_rsf_train, y_rsf_test = train_test_split(X_rsf, y_rsf, test_size=0.2, random_state=42)

In [102]:
rsf = RSF(
    n_estimators=700,
    min_samples_split=10,
    min_samples_leaf=15,
    max_features="sqrt",
    n_jobs=-1,
    random_state=42
)

rsf.fit(X_rsf_train, y_rsf_train)

,n_estimators,700
,max_depth,None
,min_samples_split,10
,min_samples_leaf,15
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,bootstrap,True
,oob_score,False
,n_jobs,-1
,random_state,42


In [103]:
risk_rsf_train = rsf.predict(X_rsf_train)
risk_rsf_test = rsf.predict(X_rsf_test)

cindex_train = c_index(
    y_rsf_train["event"], y_rsf_train["time"], risk_rsf_train
)

cindex_test = c_index(
    y_rsf_test["event"], y_rsf_test["time"], risk_rsf_test
)
print("C-index Train:", cindex_train)
print("C-index Test:", cindex_test)

C-index Train: (np.float64(0.8294945405908469), np.int64(162493), np.int64(32914), np.int64(1226), np.int64(1071))
C-index Test: (np.float64(0.8018707482993197), np.int64(8444), np.int64(2054), np.int64(86), np.int64(60))


#### RSF GENE EXP ONLY

In [104]:
X_rsf_gene = preprocessor_rsf_gene.fit_transform(df_rsf)
X_rsf_gene_feature_names = preprocessor_rsf_gene.get_feature_names_out()


if sparse.issparse(X_rsf_gene):
    X_rsf_gene = X_rsf_gene.toarray()

X_rsf_gene_df = pd.DataFrame(
    X_rsf_gene,
    columns=X_rsf_gene_feature_names,
    index=df_rsf.index
)
X_rsf_gene_df['time'] = df_rsf['time']
X_rsf_gene_df['event'] = df_rsf['event']

y_rsf_gene = np.array(
    [(bool(e), t) for e, t in zip(X_rsf_gene_df["event"], X_rsf_gene_df["time"])],
    dtype=[("event", bool), ("time", float)]
)

In [105]:
X_rsf_gene_train, X_rsf_gene_test, y_rsf_gene_train, y_rsf_gene_test = train_test_split(X_rsf_gene, y_rsf_gene, test_size=0.2, random_state=42)

In [106]:
rsf_gene = RSF(
    n_estimators=700,
    min_samples_split=10,
    min_samples_leaf=15,
    max_features="sqrt",
    n_jobs=-1,
    random_state=42
)

rsf_gene.fit(X_rsf_gene_train, y_rsf_gene_train)

,n_estimators,700
,max_depth,None
,min_samples_split,10
,min_samples_leaf,15
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,bootstrap,True
,oob_score,False
,n_jobs,-1
,random_state,42


In [107]:
risk_rsf_gene_train = rsf_gene.predict(X_rsf_gene_train)
risk_rsf_gene_test = rsf_gene.predict(X_rsf_gene_test)

cindex_gene_train = c_index(
    y_rsf_gene_train["event"], y_rsf_gene_train["time"], risk_rsf_gene_train
)

cindex_gene_test = c_index(
    y_rsf_gene_test["event"], y_rsf_gene_test["time"], risk_rsf_gene_test
)
print("C-index Train:", cindex_gene_train)
print("C-index Test:", cindex_gene_test)

C-index Train: (np.float64(0.9411670472402903), np.int64(184742), np.int64(11246), np.int64(645), np.int64(1071))
C-index Test: (np.float64(0.9188869992441421), np.int64(9711), np.int64(844), np.int64(29), np.int64(60))


#### RSF GENE EXP + DIAGNOSES (MULTIMODAL)

In [108]:
X_rsf_multi= preprocessor_rsf_multi.fit_transform(df_rsf)
X_rsf_multi_feature_names = preprocessor_rsf_multi.get_feature_names_out()


if sparse.issparse(X_rsf_multi):
    X_rsf_multi = X_rsf_multi.toarray()

X_rsf_multi_df = pd.DataFrame(
    X_rsf_multi,
    columns=X_rsf_multi_feature_names,
    index=df_rsf.index
)
X_rsf_multi_df['time'] = df_rsf['time']
X_rsf_multi_df['event'] = df_rsf['event']

y_rsf_multi = np.array(
    [(bool(e), t) for e, t in zip(X_rsf_multi_df["event"], X_rsf_multi_df["time"])],
    dtype=[("event", bool), ("time", float)]
)

In [109]:
X_rsf_multi_train, X_rsf_multi_test, y_rsf_multi_train, y_rsf_multi_test = train_test_split(X_rsf_multi, y_rsf_multi, test_size=0.2, random_state=42)

In [110]:
rsf_multi = RSF(
    n_estimators=700,
    min_samples_split=10,
    min_samples_leaf=15,
    max_features="sqrt",
    n_jobs=-1,
    random_state=42
)

rsf_multi.fit(X_rsf_multi_train, y_rsf_multi_train)

,n_estimators,700
,max_depth,None
,min_samples_split,10
,min_samples_leaf,15
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,bootstrap,True
,oob_score,False
,n_jobs,-1
,random_state,42


In [111]:
risk_rsf_multi_train = rsf_multi.predict(X_rsf_multi_train)
risk_rsf_multi_test = rsf_multi.predict(X_rsf_multi_test)

cindex_multi_train = c_index(
    y_rsf_multi_train["event"], y_rsf_multi_train["time"], risk_rsf_multi_train
)

cindex_multi_test = c_index(
    y_rsf_multi_test["event"], y_rsf_multi_test["time"], risk_rsf_multi_test
)
print("C-index Train:", cindex_multi_train)
print("C-index Test:", cindex_multi_test)

C-index Train: (np.float64(0.9415077835358256), np.int64(184809), np.int64(11179), np.int64(645), np.int64(1071))
C-index Test: (np.float64(0.918603552532124), np.int64(9708), np.int64(847), np.int64(29), np.int64(60))


### TRAD ML

In [112]:
smote = SMOTE(random_state=42)

In [113]:
rfc = RFC()

In [114]:
X_gene_train, X_gene_test, y_gene_train, y_gene_test = train_test_split(X_gene, y_survival, stratify=y_survival, test_size=0.3, random_state=42)
# Apply SMOTE
X_gene_train_res, y_gene_train_res = smote.fit_resample(X_gene_train, y_gene_train)

NameError: name 'X_gene' is not defined

In [ ]:
pipeline_gene = Pipeline(steps=[
    ('preprocessor_gene', preprocessor_gene),
    ('classifier', RFC(n_estimators=100, random_state=42))
])

pipeline_gene.fit(X_gene_train, y_gene_train)
pass

In [ ]:
pipeline_gene.score(X_gene_test, y_gene_test)

In [ ]:
"""
X_trans = pipeline_gene.named_steps[
    'preprocessor_gene'
].transform(X_gene_train)

feature_names = pipeline_gene.named_steps[
    'preprocessor_gene'
].get_feature_names_out()

X_trans_df = pd.DataFrame(
    X_trans,
    columns=feature_names,
    index=X_gene_train.index
)

print("TRANSFORMED")
print(X_trans_df.iloc[:, :6].head())
"""

#### GENES

In [ ]:
model_gene = rfc.fit(X_gene_train_res, y_gene_train_res)
model_gene.score(X_gene_test, y_gene_test)

#### BASE

In [ ]:
X_base_train, X_base_test, y_base_train, y_base_test = train_test_split(X_base, y_survival, stratify=y_survival, test_size=0.2, random_state=42)

In [ ]:
# Apply SMOTE
X_base_train_res, y_base_train_res = smote.fit_resample(X_base_train, y_base_train)

In [ ]:
model_base = rfc.fit(X_base_train_res, y_base_train_res)

In [ ]:
model_base.score(X_base_test, y_base_test)

#### TREATMENT INCLUDED

In [ ]:
X_treat_train, X_treat_test, y_treat_train, y_treat_test = train_test_split(X_treat, y_treat, test_size=0.2, random_state=42)

In [ ]:
model_treat = rfc.fit(X_treat_train, y_treat_train)

In [ ]:
len(gene_columns)

In [ ]:
df['survival'].value_counts()

# EXTERNAL VALIDATION

## METABRIC VIA cBIOPORTAL

In [115]:
edf = pd.read_csv("brca_mbcproject_2022/data_mrna_seq_v2_rsem.txt", sep="\t")

In [119]:
diagnoses_categorical_columns

['diagnoses.ajcc_pathologic_m',
 'diagnoses.ajcc_pathologic_n',
 'diagnoses.ajcc_pathologic_t',
 'diagnoses.classification_of_tumor',
 'diagnoses.icd_10_code',
 'diagnoses.laterality',
 'diagnoses.metastasis_at_diagnosis',
 'diagnoses.method_of_diagnosis',
 'diagnoses.morphology',
 'diagnoses.prior_malignancy',
 'diagnoses.prior_treatment']

In [120]:
diagnoses_numerical_columns

['age_missing', 'diagnoses.diagnosis_is_primary_disease']